In [46]:
import re
import os
import tkinter as tk
from tkinter import filedialog
class PHPToUMLConverter:
    @staticmethod
    def parse_attributes(line, current_element):
        visibility_match = re.match(r'\s*(public|protected|private)\s+\$(\w+)(?:\s*=\s*([^;]+))?\s*;', line)
        if visibility_match:
            visibility, attribute_name, default_value = visibility_match.groups()
            attribute_type = 'mixed'

            # Déduire le type de l'attribut en regardant le type initial lors de la déclaration
            if default_value:
                if default_value.isnumeric():
                    attribute_type = 'int'
                elif default_value.lower() in ('true', 'false'):
                    attribute_type = 'bool'
                elif default_value[0] in ('"', "'"):
                    attribute_type = 'string'
                elif default_value[0] in ('[', '{'):
                    attribute_type = 'array'

            current_element['attributes'].append({
                'visibility': visibility,
                'name': attribute_name,
                'type': attribute_type
            })

    @staticmethod
    def parse_methods(line, current_element):
        method_match = re.match(r'\s*(public|protected|private)\s+function\s+([a-zA-Z0-9_]+)\s*\(([^)]*)\)\s*[{;]?', line)
        if method_match:
            visibility, method_name, method_params = method_match.groups()
            params_with_types = [param.strip() for param in method_params.split(',')]
            is_abstract = '{abstract}' in line
            current_element['methods'].append({
                'visibility': '+' if visibility == 'public' else '-' if visibility == 'private' else '#',
                'name': method_name,
                'params': params_with_types,
                'abstract': is_abstract
            })

    @staticmethod
    def convert_php_to_uml(file_paths):
        elements = []
        interfaces = []
        inheritance_relations = []

        for file_path in file_paths:
            with open(file_path, 'r') as file:
                code = file.read()

            current_class = None
            current_interface = None

            for line in code.split('\n'):
                interface_match = re.match(r'interface\s+([a-zA-Z0-9_]+)\s*(?:extends\s+([a-zA-Z0-9_,\s]+))?\s*\{?', line)
                class_match = re.match(r'class\s+([a-zA-Z0-9_]+)\s*(?:implements\s+([a-zA-Z0-9_,\s]+))?(?:\s*extends\s+([a-zA-Z0-9_]+))?\s*\{', line)

                if interface_match:
                    current_interface = {'name': interface_match.group(1), 'extends': [], 'attributes': [], 'methods': []}
                    interfaces.append(current_interface)
                elif class_match:
                    name, implements, extends = class_match.groups()
                    current_class = {'name': name, 'implements': implements.split(', ') if implements else [], 'extends': extends, 'attributes': [], 'methods': []}
                    elements.append(current_class)

                if current_class:
                    PHPToUMLConverter.parse_attributes(line, current_class)
                    PHPToUMLConverter.parse_methods(line, current_class)

                if current_interface:
                    PHPToUMLConverter.parse_methods(line, current_interface)

            for element in elements:
                if element['extends']:
                    inheritance_relations.extend(f'{parent} <|-- {element["name"]}' for parent in element['extends'])
                if element['implements']:
                    inheritance_relations.extend(f'{interface} <|.. {element["name"]}' for interface in element['implements'])

        uml_template = """
        @startuml

        skinparam classAttributeIconSize 0

        {interfaces}
        {elements}
        {inheritance_relations}
        @enduml
        """

        interface_template = """
        interface {interface_name} {{
        {methods}
        }}
        """

        class_template = """
        class {class_name} {{
        {attributes}
        {methods}
        }}
        """

        file_path = os.path.join("./plantUml", "shema.puml")
        existing_content = ""

        if os.path.exists(file_path):
            with open(file_path, 'r') as existing_file:
                existing_content = existing_file.read()

        for interface_ in interfaces:
            interface_name = interface_['name']
            methods = '\n'.join(f'  {method["visibility"]} {method["name"]}({",".join(method["params"])}){"" if not method["abstract"] else " {abstract}"}' for method in interface_['methods'])
            new_uml = interface_template.format(interface_name=interface_name, methods=methods)
            existing_content += new_uml

        for element in elements:
            class_name = element['name']
            attributes = '\n'.join(f'  - {attribute["name"]} : {attribute["type"]}' for attribute in element['attributes'])
            methods = '\n'.join(f'  {method["visibility"]} {method["name"]}({",".join(method["params"])}){"" if not method["abstract"] else " {abstract}"}' for method in element['methods'])
            new_uml = class_template.format(class_name=class_name, attributes=attributes, methods=methods)
            existing_content += new_uml

        existing_content += '\n'.join(inheritance_relations)

        with open(file_path, 'w') as file:
            file.write(existing_content)

if __name__ == '__main__':
    root = tk.Tk()
    root.withdraw()
    file_paths = filedialog.askopenfilenames(
        title="Sélectionner les fichiers à convertir en PlantUML",
        filetypes=[("Fichiers PHP et Python", "*.php;*.py"), ("Tous les fichiers", "*.*")]
    )
    PHPToUMLConverter.convert_php_to_uml(file_paths)


In [42]:
class UMLToPHPConverter:
    @staticmethod
    def convert_class(class_name, attributes, methods):
        php_code = f"class {class_name} {{\n"
        php_code += UMLToPHPConverter.convert_attributes(attributes)
        php_code += UMLToPHPConverter.convert_methods(methods)
        php_code += "}\n"
        return php_code

    @staticmethod
    def convert_attributes(attributes):
        return "\n".join([f"  private ${attr['name']};\n" for attr in attributes])

    @staticmethod
    def convert_methods(methods):
        return "\n".join([f"  public function {method['name']}({', '.join(method['params'])}) {{\n    // TODO: Implement {method['name']} method\n  }}\n" for method in methods])

# Exemple d'utilisation
uml_data = {
    'name': 'Person',
    'attributes': [{'name': 'name', 'type': 'string'}],
    'methods': [{'name': 'sayHello', 'params': []}]
}

php_code = UMLToPHPConverter.convert_class(uml_data['name'], uml_data['attributes'], uml_data['methods'])
print(php_code)


class Person {
  private $name;
  public function sayHello() {
    // TODO: Implement sayHello method
  }
}

